In [40]:

import numpy as np
import pandas as pd
import seaborn as sns

In [41]:
df=sns.load_dataset('tips')

In [42]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [43]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [44]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [45]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [46]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [47]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [48]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [49]:
df.time.unique()

array([0, 1])

In [50]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [51]:
x=df.drop(labels=['time'],axis=1)
y=df.time

In [52]:
x

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [53]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [54]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [55]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()


In [56]:
y_pred=model.fit(x_train,y_train)

ValueError: could not convert string to float: 'Male'

there is an error beacuse we didn't convert the categorical data in the input hence we make use ok data pipeline

data pipeline is an automated way of performing all the preprocessing of data before splitting onto train and test data where we normalize the data, we standardize the data, we perform all the encoding, and handling of missing data 

In [ ]:
from sklearn.pipeline import Pipeline

#for missing handeling missing values
from sklearn.impute import SimpleImputer

#for feature scaling
from sklearn.preprocessing import StandardScaler

# for encoding 
from sklearn.preprocessing import OneHotEncoder

#for combining multiple pipelines(to transform the columns)
from sklearn.compose import ColumnTransformer


In [ ]:
x.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [ ]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','size','tip']

# Automation of Feature Engineering

In [ ]:
## Numerical pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),# missing values handeled
        ('scaler',StandardScaler()),#feature scaling
    ]
)

#categorical pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),# missing values handeled
        ('onehotencoding',OneHotEncoder()),#cat to numerical
    ]
)


In [58]:
#combining the above pipelines because the data will always containg both cat and num features
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),#used defined name,calling the above function,giving the feature names
    ('cal_pipeline',cat_pipeline,categorical_cols)
])

preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'size', 'tip']),
                                ('cal_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoding',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [57]:
x_train=preprocessor.fit_transform(x_train)
x_test=preprocessor.transform(x_test)

# Model Training Automation

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [70]:
## Model Training Automation
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier()

}

In [67]:
from sklearn.metrics import accuracy_score

In [68]:
def evaluate_model(x_train,y_train,x_test,y_test,models):

    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(x_train,y_train)


        # Predict Testing data
        y_test_pred =model.predict(x_test)

        # Get accuracy for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score



    return report


In [71]:
evaluate_model(x_train,y_train,x_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [72]:
classfier=RandomForestClassifier()

In [73]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [74]:
from sklearn.model_selection import RandomizedSearchCV

In [75]:
cv=RandomizedSearchCV(classfier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.949 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.949 total time=   0.2s
[CV 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [76]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 3, 'criterion': 'gini'}